In [98]:
from imports import pd, time
import numpy as np
import yfinance as source
from itertools import islice

In [99]:
# URLs for the NASDAQ Trader Symbol Directory files
# List is updated everyday.
nasdaq_url = "ftp://ftp.nasdaqtrader.com/SymbolDirectory/nasdaqlisted.txt"
# other_url = "ftp://ftp.nasdaqtrader.com/SymbolDirectory/otherlisted.txt"

# Load NASDAQ-listed stocks
nasdaq_data = pd.read_csv(nasdaq_url, sep="|")
# Remove the last row (footer info)
nasdaq_data = nasdaq_data[:-1]

# # Load other-listed stocks (including NYSE, AMEX, etc.)
# other_data = pd.read_csv(other_url, sep="|")
# # Filter for NYSE stocks
# nyse_stocks = other_data[other_data["Exchange"] == "N"]

In [ ]:
#  Filter for companies that are not bankrupt, delisted, etc.
normal_cmps = nasdaq_data.where(nasdaq_data["Financial Status"] == "N").dropna()
normal_cmps.index = range(len(normal_cmps))
normal_cmps

In [ ]:
nasdaq_tickers = normal_cmps[["Symbol", "Security Name"]]
nasdaq_tickers.loc[:, "Exchange"] = "NASDAQ"
nasdaq_tickers

In [107]:
data_points = ['symbol', 'shortName', 'previousClose', 'open', 'dayLow', 'dayHigh', 'regularMarketPreviousClose',
               'regularMarketOpen', 'regularMarketDayLow', 'regularMarketDayHigh', 'dividendRate', 'dividendYield',
              'fiveYearAvgDividendYield', 'beta', 'trailingPE', 'forwardPE', 'volume',  'marketCap',
              'fiftyTwoWeekLow', 'fiftyTwoWeekHigh', 'priceToSalesTrailing12Months', 'fiftyDayAverage', 
              'twoHundredDayAverage', 'profitMargins', 'bookValue', 'priceToBook', 'earningsQuarterlyGrowth', 
              'netIncomeToCommon', 'trailingEps', 'forwardEps', 'enterpriseToRevenue', 'enterpriseToEbitda', '52WeekChange',
              'ebitda', 'totalDebt', 'quickRatio', 'currentRatio', 'totalRevenue', 'debtToEquity', 'revenuePerShare', 'returnOnAssets', 
              'returnOnEquity', 'grossProfits', 'freeCashflow', 'operatingCashflow', 'earningsGrowth', 'revenueGrowth', 'grossMargins', 
              'ebitdaMargins', 'operatingMargins', 'trailingPegRatio']
def download_data(ticker):
    try: 
        stock = source.Ticker(ticker)
        stock_info = stock.info
        print(f"Fetched data for {ticker}")
        return [stock_info.get(key, None) for key in data_points]
    except Exception as e:
        print(f"Could not get data for {ticker}")


In [ ]:
stocksData = []
batch_size = 50
m = len(list(nasdaq_tickers["Symbol"]))
for b in range(0, m, batch_size):
    for i in range(batch_size):
        stocksData.append(download_data(nasdaq_tickers["Symbol"][b+i]))
    time.sleep(1)

In [136]:
stocksData = [i for i in stocksData if i!=None]
print(f"{len(stocksData)} stocks data fetched!")

stocksData_df = pd.DataFrame(stocksData, columns=data_points)
stocksData_df = stocksData_df.dropna(subset=["symbol"])
stocksData_df = stocksData_df.replace([np.nan], 0.0)
stocksData_df.columns = [i.capitalize() for i in stocksData_df.columns]
stocksData_df.to_csv("stocksData.csv", index=False)
print("Saved data to csv file.")

4157 stocks data fetched!
Saved data to csv file.


Index(['Symbol', 'Shortname', 'Previousclose', 'Open', 'Daylow', 'Dayhigh',
       'Regularmarketpreviousclose', 'Regularmarketopen',
       'Regularmarketdaylow', 'Regularmarketdayhigh', 'Dividendrate',
       'Dividendyield', 'Fiveyearavgdividendyield', 'Beta', 'Trailingpe',
       'Forwardpe', 'Volume', 'Marketcap', 'Fiftytwoweeklow',
       'Fiftytwoweekhigh', 'Pricetosalestrailing12months', 'Fiftydayaverage',
       'Twohundreddayaverage', 'Profitmargins', 'Bookvalue', 'Pricetobook',
       'Earningsquarterlygrowth', 'Netincometocommon', 'Trailingeps',
       'Forwardeps', 'Enterprisetorevenue', 'Enterprisetoebitda',
       '52weekchange', 'Ebitda', 'Totaldebt', 'Quickratio', 'Currentratio',
       'Totalrevenue', 'Debttoequity', 'Revenuepershare', 'Returnonassets',
       'Returnonequity', 'Grossprofits', 'Freecashflow', 'Operatingcashflow',
       'Earningsgrowth', 'Revenuegrowth', 'Grossmargins', 'Ebitdamargins',
       'Operatingmargins', 'Trailingpegratio'],
      dtype='obje